USER REVIEWS

In [ ]:
# se importan las librerias que se van a usar 
import gzip
import ast
import chardet
import pandas as pd
import nltk

Se accede a la informacion del archivo comprimido

In [161]:
file_path_user_reviews = '../datasets/user_reviews.json.gz'
lista=[]
with gzip.open(file_path_user_reviews, 'rt', encoding='MacRoman') as archivo:
    for line in archivo.readlines():
        lista.append(ast.literal_eval(line))

user_review = pd.DataFrame(lista)
user_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


se procede a desanidar la columna reviews que se necesitara mas adelante pare el proceso 

In [162]:
# Utiliza explode para duplicar las filas generando un diccionario por cada diccionario en la lista
explode_df_user_review = user_review.explode('reviews')

# Utiliza json_normalize para expandir los diccionarios en nuevas columnas
expanded_reviews = pd.json_normalize(explode_df_user_review['reviews'])

# Combina el DataFrame original con el DataFrame generado
explode_df_user_review = explode_df_user_review.drop(["reviews"], axis=1).merge(expanded_reviews, left_index=True, right_index=True)
explode_df_user_review.info()


<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   funny        59314 non-null  object
 3   posted       59314 non-null  object
 4   last_edited  59314 non-null  object
 5   item_id      59314 non-null  object
 6   helpful      59314 non-null  object
 7   recommend    59314 non-null  object
 8   review       59314 non-null  object
dtypes: object(9)
memory usage: 4.5+ MB


In [165]:
explode_df_user_review[explode_df_user_review.duplicated()]

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,evcentric,http://steamcommunity.com/id/evcentric,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,blackman retribution
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,blackman retribution
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,blackman retribution
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,‡πÄ‡∏£‡∏∑‡πà‡∏≠‡∏á‡∏Ç‡∏≠‡∏á‡∏†‡∏≤‡∏û 10/10 ‡...


se procede a eliminar las columnas que no son necesarias para el procedimiento

In [ ]:
explode_df_user_review.drop(columns=['user_url','funny','helpful','last_edited'],inplace=True)#esta linea elimina las columnas que no necesitamos

Se realiza el analisis de sentimiento solicitado con NLP

In [ ]:

from nltk.sentiment import SentimentIntensityAnalyzer

# Descargar recursos de NLTK 
nltk.download('vader_lexicon')

# Crear una instancia de SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Crear una función para asignar el análisis de sentimiento
def assign_sentiment_analysis(review):
    if pd.isna(review):  # Si la reseña está ausente
        return 1  # Valor neutral
    else:
        # Realizar análisis de sentimiento con vaderSentiment
        compound_score = sia.polarity_scores(review)['compound']

        if compound_score >= 0.05:
            return 2  # Positivo
        elif compound_score <= -0.05:
            return 0  # Negativo
        else:
            return 1  # Neutral

# Aplicar la función a la columna 'review' y crear la nueva columna 'sentiment_analysis'
explode_df_user_review['sentiment_analysis'] = explode_df_user_review['review'].apply(assign_sentiment_analysis)

# Reemplazar la columna original 'review' con la nueva 'sentiment_analysis'
explode_df_user_review.drop(columns=['review'], inplace=True)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Willc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


             user_id                    posted item_id recommend  \
0  76561197970982479  Posted November 5, 2011.    1250      True   
0  76561197970982479  Posted November 5, 2011.    1250      True   
0  76561197970982479  Posted November 5, 2011.    1250      True   
1            js41637     Posted July 15, 2011.   22200      True   
1            js41637     Posted July 15, 2011.   22200      True   

   sentiment_analysis  
0                   2  
0                   2  
0                   2  
1                   2  
1                   2  


In [ ]:
explode_df_user_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59333 non-null  object
 1   posted              59314 non-null  object
 2   item_id             59314 non-null  object
 3   recommend           59314 non-null  object
 4   sentiment_analysis  59333 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.7+ MB


Se verifica si en el dataframe hay valores nulos

In [ ]:
explode_df_user_review.isnull().sum()

user_id               0
posted                0
item_id               0
recommend             0
sentiment_analysis    0
dtype: int64

In [ ]:
explode_df_user_review[explode_df_user_review.isnull().any(axis=1)]

,user_id,posted,item_id,recommend,sentiment_analysis
137,2xDelorean,NaN,NaN,NaN,1
137,2xDelorean,NaN,NaN,NaN,1
137,2xDelorean,NaN,NaN,NaN,1
137,2xDelorean,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1


ya que las filas estan completamente vacias se procede a eliminarlas

In [ ]:
explode_df_user_review.dropna(inplace=True)

Por ultimo despuesde de haber realizado las transformaciones solicitadas y las que se consideraron necesarias se procede a exportar a un csv

In [ ]:
explode_df_user_review.to_csv('../datasets/user_review.csv',index=False)